In [104]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go

In [105]:
usb_path = "../../../"

In [ ]:
#load campaign finance for candidates
columns_campfin22_cands22 = ['cycle', 'feccandid', 'cid', 'firstlastp', 'party', 'distidrunfor', 
                    'distidcurr', 'currcand', 'cyclecand', 'crpico', 'recipcode', 
                    'nopacs']
columns_campfin22_cands22 = dict(enumerate(columns_campfin22_cands22))

df_cand = pd.read_csv(usb_path + 'CampaignFin22/cands22.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'])
df_cand = df_cand.rename(columns=columns_campfin22_cands22)
df_cand = df_cand.applymap(lambda x: x.strip('|') if isinstance(x, str) else x)
df_cand.head()

,cycle,feccandid,cid,firstlastp,party,distidrunfor,distidcurr,currcand,cyclecand,crpico,recipcode,nopacs
0,2022,H0AK00105,N00039029,Thomas Lamb (3),3,AK01,,,,,3N,
1,2022,H0AL01055,N00044245,Jerry Carl (R),R,AL01,AL01,Y,Y,I,RW,
2,2022,H0AL01063,N00044288,Wes Lambert (R),R,AL01,,,,,RN,
3,2022,H0AL01097,N00044750,James Averhart (D),D,AL01,,,,,DN,
4,2022,H0AL02087,N00030768,Martha Roby (R),R,AL02,,,,,RN,


In [ ]:
#load campaign finance for pacs
columns_campfin22_pacs22 = ['cycle', 'fecrecno', 'pacid', 'cid', 'amount', 'date', 'realcode', 
                            'type', 'di', 'feccandid']
columns_campfin22_pacs22 = dict(enumerate(columns_campfin22_pacs22))

df_pacs = pd.read_csv(usb_path + 'CampaignFin22/pacs22.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'])

df_pacs = df_pacs.rename(columns=columns_campfin22_pacs22)
df_pacs = df_pacs.applymap(lambda x: x.strip('|') if isinstance(x, str) else x)
df_pacs.head()

,cycle,fecrecno,pacid,cid,amount,date,realcode,type,di,feccandid
0,2022,1011020220264924462,C00406124,N00029513,500,10/18/2021,B4000,24K,D,H8IN07184
1,2022,1011320230265253020,C00032979,N00041569,-2500,11/21/2022,Z9600,24K,D,H8IL14174
2,2022,1011320230265254779,C00032979,N00041511,-5000,11/18/2022,Z9600,24K,D,H8MN08043
3,2022,1011320230265254798,C00032979,N00035575,5000,11/03/2022,Z9600,24K,D,H4FL15155
4,2022,1011320230265254809,C00032979,N00041511,4000,11/03/2022,Z9600,24K,D,H8MN08043


In [ ]:
#load campaign finance for committees
columns_campfin22_cmtes22 = ['cycle', 'cmteid', 'pacshort', 'affiliate', 'ultorg', 'recipid', 
                    'recipcode', 'feccandid', 'party', 'primcode', 'source', 'sensitive',
                    'foreign', 'active']
columns_campfin22_cmtes22 = dict(enumerate(columns_campfin22_cmtes22))
usb_path = "../../../"
df_cmt = pd.read_csv(usb_path + 'CampaignFin22/cmtes22.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'])
df_cmt = df_cmt.rename(columns=columns_campfin22_cmtes22)
df_cmt = df_cmt.applymap(lambda x: x.strip('|') if isinstance(x, str) else x)
df_cmt.head()

,cycle,cmteid,pacshort,affiliate,ultorg,recipid,recipcode,feccandid,party,primcode,source,sensitive,foreign,active
0,2022,C00000059,Hallmark Cards,NaN,Hallmark Cards,C00000059,PB,NaN,NaN,C1400,WAA21,Y,0,1
1,2022,C00000422,American Medical Assn,American Medical Assn,American Medical Assn,C00000422,PB,NaN,NaN,H1100,WAA21,n,0,1
2,2022,C00000489,Teamsters Local 886,Teamsters Union,Teamsters Union,C00000489,PL,NaN,,LT300,WAA21,n,0,1
3,2022,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMIT...,NaN,NaN,C00000547,NaN,NaN,U,NaN,NaN,NaN,0,0
4,2022,C00000729,American Dental Assn,American Dental Assn,American Dental Assn,C00000729,PB,NaN,NaN,H1400,WAA21,n,0,1


In [ ]:
#find candidates on the ways and means committee
ways_and_means_members = ["Earl Blumenauer", "Don Beyer"]
df_cand['firstlastp'] = df_cand['firstlastp'].apply(lambda x: ' '.join(x.split()[:2]))
df_cand_filt = df_cand[df_cand['firstlastp'].isin(ways_and_means_members)]
df_cand_filt.head()

,cycle,feccandid,cid,firstlastp,party,distidrunfor,distidcurr,currcand,cyclecand,crpico,recipcode,nopacs
5031,2022,H4VA08224,N00036018,Don Beyer,D,VA08,VA08,Y,Y,I,DW,
5294,2022,H6OR03064,N00007727,Earl Blumenauer,D,OR03,OR03,Y,Y,I,DW,


In [111]:
#Ways and Means PACs
df_pacs_wm = df_pacs.merge(df_cand_filt.set_index('feccandid'), on=['feccandid', 'cid'])
df_pacs_wm.head()

,cycle,fecrecno,pacid,cid,amount,date,realcode,type,di,feccandid,firstlastp,party
0,2022,1081920220264974933,C00024521,N00036018,3000,07/29/2022,K1100,24K,D,H4VA08224,Don Beyer,D
1,2022,4010420231646811157,C00433482,N00036018,1000,11/01/2022,C5400,24K,D,H4VA08224,Don Beyer,D
2,2022,4010420231665460861,C00325092,N00036018,1000,10/27/2022,D5000,24K,D,H4VA08224,Don Beyer,D
3,2022,4010520231666136221,C00004036,N00036018,2500,09/27/2022,LG300,24K,D,H4VA08224,Don Beyer,D
4,2022,4010920231666588681,C00318311,N00036018,500,11/07/2022,L1100,24K,D,H4VA08224,Don Beyer,D


In [ ]:
#merge pac information with candidate personal identifiers
df_cmt_filt = df_cmt[["cmteid", "pacshort", "affiliate", "ultorg", "primcode"]]
df_pacs_wm_cmt = df_pacs_wm.merge(df_cmt_filt.set_index('cmteid'), left_on='pacid', right_on='cmteid')
df_pacs_wm_cmt["quarter"] = pd.to_datetime(df_pacs_wm_cmt["date"]).dt.to_period("Q")
df_pacs_wm_cmt["year"] = pd.to_datetime(df_pacs_wm_cmt["date"]).dt.year
df_pacs_wm_cmt[["pacid", "pacshort", "ultorg", "cid", "firstlastp", "party", "date", "year", "quarter", "amount"]]

,pacid,pacshort,ultorg,cid,firstlastp,party,date,year,quarter,amount
0,C00024521,American Assn for Justice,American Assn for Justice,N00036018,Don Beyer,D,07/29/2022,2022,2022Q3,3000
1,C00024521,American Assn for Justice,American Assn for Justice,N00036018,Don Beyer,D,04/13/2021,2021,2021Q2,1000
2,C00024521,American Assn for Justice,American Assn for Justice,N00036018,Don Beyer,D,05/05/2021,2021,2021Q2,1000
3,C00024521,American Assn for Justice,American Assn for Justice,N00036018,Don Beyer,D,09/08/2021,2021,2021Q3,3000
4,C00024521,American Assn for Justice,American Assn for Justice,N00036018,Don Beyer,D,09/08/2021,2021,2021Q3,2000
...,...,...,...,...,...,...,...,...,...,...
735,C00032979,Teamsters Union,Teamsters Union,N00007727,Earl Blumenauer,D,11/30/2021,2021,2021Q4,5000
736,C00007070,Texas Instruments,Texas Instruments,N00007727,Earl Blumenauer,D,10/11/2022,2022,2022Q4,2000
737,C00164145,USAA,USAA,N00007727,Earl Blumenauer,D,09/26/2022,2022,2022Q3,2500
738,C00017525,American Nurses Assn,American Nurses Assn,N00007727,Earl Blumenauer,D,10/06/2022,2022,2022Q4,2500


In [ ]:
#group pac committee contributions by candidate and quarter
df_pacs_wm_cmt.groupby(['ultorg','firstlastp', "year", "quarter"])["amount"].sum().reset_index().sort_values(by='amount', ascending=False)

,ultorg,firstlastp,year,quarter,amount
613,SpaceX,Don Beyer,2021,2021Q2,10000
282,ESOP Assn,Don Beyer,2022,2022Q2,10000
566,PeopleForBikes,Earl Blumenauer,2021,2021Q3,10000
357,Humane Society of the US,Earl Blumenauer,2022,2022Q4,7599
203,Blue Cross/Blue Shield,Earl Blumenauer,2021,2021Q3,6000
...,...,...,...,...,...
135,American Hotel & Lodging Assn,Don Beyer,2022,2022Q4,0
337,Hawkeye 360,Don Beyer,2022,2022Q3,-100
50,America's Communications Assn,Don Beyer,2021,2021Q2,-1000
310,Friedkin Group,Don Beyer,2022,2022Q4,-1500


In [114]:
# Dataset description:
# https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20lob_lobbying.htm
columns_lob_lobbying = ['uniqid', 'registrant_raw', 'registrant', 'isfirm', 
                     'client_raw', 'client', 'ultorg', 'amount', 'cid', 'catcode'
                     'source', 'self', 'includensfs', 'use', 'ind', 'year',
                     'type', 'typelong', 'affiliate']

columns_lob_lobbying = dict(enumerate(columns_lob_lobbying))

df_lobbying = pd.read_csv('../../data/open_secrets/Lobby/lob_lobbying.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'], encoding='ISO-8859-1')
df_lobbying = df_lobbying.rename(columns=columns_lob_lobbying)
df_lobbying = df_lobbying.applymap(lambda x: x.strip('|') if isinstance(x, str) else x)

/var/folders/g1/9sc25jzn7fdgvxb50169_b9m0000gn/T/ipykernel_63716/3327076248.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lobbying = pd.read_csv('../../data/open_secrets/Lobby/lob_lobbying.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'], encoding='ISO-8859-1')


In [ ]:
#load lobbying firms to agencies
columns_lob_agency = ['uniqid', 'agencyid', 'agency']
columns_lob_agency = dict(enumerate(columns_lob_agency))

df_agency = pd.read_csv('../../data/open_secrets/Lobby/lob_agency.csv', on_bad_lines='skip', sep=',', header=None, na_values=['N/A', 'NA'], encoding='ISO-8859-1')
df_agency = df_agency.rename(columns=columns_lob_agency)
df_agency = df_agency.applymap(lambda x: x.strip('|') if isinstance(x, str) else x)

In [ ]:
#find how many companies who finance campaigns also lobby
companies_in_both = (set(df_lobbying["registrant"]).intersection(set(df_pacs_wm_cmt["ultorg"])))
companies_no_lobbying = set(df_pacs_wm_cmt["ultorg"]).difference(set(df_lobbying["registrant"]))
print(len(companies_no_lobbying))
print(len(companies_in_both))
print("Percentage of companies that finance campaigns who also lobby: ",)
print(len(companies_in_both) / (len(companies_in_both) + len(companies_no_lobbying)))

92
232
Percentage of companies that finance campaigns who also lobby: 
0.7160493827160493


In [ ]:
df_lobbying_filt = df_lobbying[["uniqid", "registrant", "client", "ultorg", "amount", "cid", "year", "type"]]
df_lobbying_filt['quarter'] =  df_lobbying_filt['year'].astype(str) + df_lobbying_filt['type'].str[:2].str.capitalize()
df_lobbying_filt['quarter'] = pd.to_datetime(df_lobbying_filt['quarter'], errors='coerce').dt.to_period('Q-DEC')

/var/folders/g1/9sc25jzn7fdgvxb50169_b9m0000gn/T/ipykernel_63716/3375020338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lobbying_filt['quarter'] =  df_lobbying_filt['year'].astype(str) + df_lobbying_filt['type'].str[:2].str.capitalize()
/var/folders/g1/9sc25jzn7fdgvxb50169_b9m0000gn/T/ipykernel_63716/3375020338.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_lobbying_filt['quarter'] = pd.to_datetime(df_lobbying_filt['quarter'], errors='coerce').dt.to_period('Q-DEC')
/var/folders/g1/9sc25jzn7fdgvxb50169_b9m0000gn/T/ipykernel_63716/3375020338.py:3: SettingWithCopyWarning: 
A value is trying to

,uniqid,registrant,client,ultorg,amount,cid,year,type,quarter
114,0cd09894-1c0d-4326-9f87-49757b2fc93e,MWWPR,Georgian Court University,Georgian Court University,0.0,H5100,2022,q3,2022Q3
115,68de881a-26f1-44ca-ba30-e8307cdd7993,Woolf Strategic Solutions Group,Safe House Project,Safe House Project,15000.0,Y4000,2022,q2,2022Q2
116,095f6573-3bce-4fb3-88f0-756289ef8d59,Bavarian Nordic,Bavarian Nordic,Bavarian Nordic,130000.0,H4500,2022,q1,2022Q1
117,68e4dfc7-c745-4ddf-8143-d6d3af118f6a,American Thoracic Society,American Thoracic Society,American Thoracic Society,121000.0,H1130,2022,q1,2022Q1
119,5062c9f7-7081-4919-a89b-67f24fb06958,Bramer Group,Kongsberg Defence & Aerospace,Kongsberg Gruppen,98000.0,D3000,2022,q2,2022Q2
...,...,...,...,...,...,...,...,...,...
941142,7045713b-ef0e-49fe-9737-2bf9e7c5ec4d,Empire Consulting Group,Boeing Co,Boeing Co,30000.0,T1200,2023,q1,2023Q1
941143,ad14e3fb-6cf3-4d7c-82b7-1f90a29129bf,Carpenter Strategic Consulting,Lynk Global,Lynk Global,10000.0,G5280,2022,q4,2022Q4
941144,1296e36a-0be9-4aa7-8696-1247c6840937,Farragut Partners,Animal Wellness Action,Animal Wellness Action,20000.0,J7600,2022,q4,2022Q4
941145,0854dd96-ed91-4219-99d9-b84bf06ee2f8,American Express,American Express,American Express,440000.0,F1400,2022,q4,2022Q4


In [ ]:
#Walmart example
df_lobbying_walmart = df_lobbying[df_lobbying["ultorg"] == "Walmart Inc"]
walmart_congress = df_lobbying_walmart[["uniqid", "registrant", "ultorg", "amount", "year", "type"]].merge(df_agency, on = 'uniqid')
walmart_congress['year'] = pd.to_numeric(walmart_congress['year'], errors='coerce')
walmart_congress = walmart_congress.dropna(subset=['year'])
walmart_congress['amount'] = pd.to_numeric(walmart_congress['amount'], errors='coerce')
walmart_congress = walmart_congress.dropna(subset=['amount'])
walmart_rep_lobbying = walmart_congress[(walmart_congress["year"] >= 2021) & (walmart_congress["agencyid"]== '002 ')].groupby(["registrant", "year"])["amount"].sum()

In [ ]:
#normalize amount of lobbying by number of representatives
walmart_rep_lobbying_df = walmart_rep_lobbying.reset_index()
walmart_rep_lobbying_df["amount_norm"] = walmart_rep_lobbying_df["amount"] / 435
walmart_rep_lobbying_df

,registrant,year,amount,amount_norm
0,Capitol Consulting Group LLC,2021.0,160000.0,367.816092
1,Capitol Consulting Group LLC,2022.0,160000.0,367.816092
2,Capitol Consulting Group LLC,2023.0,40000.0,91.954023
3,Capitol Counsel,2021.0,160000.0,367.816092
4,Capitol Counsel,2022.0,320000.0,735.632184
5,Capitol Counsel,2023.0,80000.0,183.908046
6,Empire Consulting Group,2021.0,200000.0,459.770115
7,Empire Consulting Group,2022.0,200000.0,459.770115
8,Empire Consulting Group,2023.0,50000.0,114.942529
9,Ferox Strategies,2021.0,160000.0,367.816092
